In [1]:
// Nuget Packages
#r "nuget: MelonChart.NET, 2.*"

#r "nuget: Microsoft.Extensions.Configuration.Json, 8.*"
#r "nuget: Microsoft.SemanticKernel, 1.*"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.*"
#r "nuget: Microsoft.SemanticKernel.Core, 1.*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.*-*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.*-*"

#r "nuget: System.Linq.Async, 6.*"


Installed Packages MelonChart.NET, 2.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.16.0 Microsoft.SemanticKernel.Connectors.OpenAI, 1.16.0 Microsoft.SemanticKernel.Core, 1.16.0 Microsoft.SemanticKernel.Plugins.Core, 1.16.0-alpha Microsoft.SemanticKernel.Plugins.Memory, 1.16.0-alpha System.Linq.Async, 6.0.1

In [2]:
// Add using statements
using System.ComponentModel;
using System.IO;
using System.Net.Http;
using System.Text.Encodings.Web;
using System.Text.Json;
using System.Text.Json.Serialization;

using MelonChart.Models;

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Memory;

using Kernel = Microsoft.SemanticKernel.Kernel;

In [3]:
// Azure OpenAI configurations
var path = Path.Combine(Directory.GetCurrentDirectory(), "appsettings.Development.json");
var config = new ConfigurationBuilder().AddJsonFile(path).Build();

var endpoint = config["Azure:OpenAI:Endpoint"];
var apiKey = config["Azure:OpenAI:ApiKey"];
var deploymentName = config["Azure:OpenAI:DeploymentName"];

In [4]:
// Build Semantic Kernel
var kernel = Kernel.CreateBuilder()
                   .AddAzureOpenAIChatCompletion(
                       deploymentName: deploymentName,
                       endpoint: endpoint,
                       apiKey: apiKey)
                   .Build();

In [5]:
// Import prompts
var prompts = kernel.ImportPluginFromPromptDirectory("Prompts/Spotify");

In [6]:
// Import codes
#!import Plugins/ChartMemories/SpotifyChartPlugin.cs

In [7]:
// Import plugins
kernel.ImportPluginFromType<SpotifyChartPlugin>();

In [8]:
// Build Semantic Memory
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0010
#pragma warning disable SKEXP0050

var memory = new MemoryBuilder()
                 .WithAzureOpenAITextEmbeddingGeneration(
                     deploymentName: "model-textembeddingada002-2",
                     endpoint: endpoint,
                     apiKey: apiKey)
                 .WithMemoryStore(new VolatileMemoryStore())
                 .Build();

In [9]:
// Add HttpClient instance.
var http = new HttpClient();


In [10]:
// Add JsonSerializerOptions instance.
var jso = new JsonSerializerOptions()
{
    WriteIndented = false,
    PropertyNamingPolicy = JsonNamingPolicy.CamelCase,
    Encoder = JavaScriptEncoder.UnsafeRelaxedJsonEscaping,
    Converters = { new JsonStringEnumConverter(JsonNamingPolicy.CamelCase) },
};

In [11]:
// Invoke the plugin - Add Spotify Chart data
await kernel.InvokeAsync(
    pluginName: nameof(SpotifyChartPlugin),
    functionName: nameof(SpotifyChartPlugin.AddChart), 
    arguments: new KernelArguments()
    {
        { "memory", memory }, 
        { "http", http },
        { "jso", jso }, 
    }
);

- Stored: aespa - Supernova
- Stored: Lee Young Ji, D.O. - Small girl (feat. D.O.)
- Stored: NewJeans - How Sweet
- Stored: ECLIPSE - Sudden Shower
- Stored: NewJeans - Supernatural
- Stored: NewJeans - Bubble Gum
- Stored: KISS OF LIFE - Sticky
- Stored: QWER - T.B.H
- Stored: aespa - Armageddon
- Stored: DAY6 - 한 페이지가 될 수 있게
- Stored: ZICO, JENNIE - SPOT!
- Stored: ILLIT - Magnetic
- Stored: IVE - HEYA
- Stored: (G)I-DLE - Fate
- Stored: TWS - plot twist
- Stored: DAY6 - 예뻤어
- Stored: Emile PARK - (Klaxon)클락션
- Stored: CAN - 천상연
- Stored: DAY6 - Welcome to the Show
- Stored: Lim Jae Beum - 이밤이지나면
- Stored: Crush - Love You With All My Heart
- Stored: RIIZE - Boom Boom Bass
- Stored: IU - Love wins all
- Stored: Lim Jae Hyun - Rhapsody of Sadness
- Stored: NewJeans - Hype Boy
- Stored: Na Yoon Kwon - 나였으면
- Stored: Han Kyung Il - 슬픈 초대장
- Stored: Standing Egg - Old Song
- Stored: IVE - I AM
- Stored: TWS - If I’m S, Can You Be My N?
- Stored: BABYMONSTER - SHEESH
- Stored: Parc Jae Ju

In [14]:
// User input
var question = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("지금 무슨 생각을 하고 계세요?");

Console.WriteLine($"User: {question}");

User: 나 너무 우울해서 빵 샀어


In [15]:
// Invoke the prompt - Get Sentiments
var sentiments = await kernel.InvokeAsync<string>(
    function: prompts["GetSentiments"],
    arguments: new KernelArguments()
    {
        { "input", question }
    });

Console.WriteLine(sentiments);

1. Sadness
2. Desperation
3. Comfort-seeking
4. Self-soothing
5. Resignation


In [16]:
// Invoke the prompt - Measure Sentiments
var measured = await kernel.InvokeAsync<string>(
    function: prompts["MeasureSentiments"],
    arguments: new KernelArguments()
    {
        { "input", question },
        { "sentiments", sentiments },
    });

Console.WriteLine(measured);

| Sentiment       | Danceability | Valence     |
| ----------------|--------------|-------------|
| Sadness         | 0.20 - 0.40  | 0.10 - 0.30 |
| Desperation     | 0.20 - 0.40  | 0.10 - 0.30 |
| Comfort-seeking | 0.30 - 0.50  | 0.40 - 0.60 |
| Self-soothing   | 0.30 - 0.50  | 0.40 - 0.60 |
| Resignation     | 0.20 - 0.40  | 0.20 - 0.40 |


In [17]:
// Invoke the prompt - Pick Primary Sentiment
var sentiment = await kernel.InvokeAsync<string>(
    function: prompts["PickPrimarySentiment"],
    arguments: new KernelArguments()
    {
        { "input", question },
        { "measured", measured },
    });

Console.WriteLine(sentiment);

```json
{
  "sentiment": "Comfort-seeking",
  "danceability": "0.30 - 0.50",
  "valence": "0.40 - 0.60"
}
```


In [18]:
// Invoke the prompt - RefineQuestion
var refined = await kernel.InvokeAsync<string>(
    function: prompts["RefineQuestion"],
    arguments: new KernelArguments()
    {
        { "input", question },
        { "sentiment", sentiment }
    });

Console.WriteLine(refined);

List all the songs with valence between 0.40 and 0.60


In [20]:
// Invoke the plugin - Find songs
var results = await kernel.InvokeAsync(
    pluginName: nameof(SpotifyChartPlugin),
    functionName: nameof(SpotifyChartPlugin.FindSongs), 
    arguments: new KernelArguments()
    {
        { "memory", memory }, 
        { "question", refined },
        { "jso", jso }, 
    }
);

var data = results.GetValue<List<ChartItem>>().Select(p => JsonSerializer.Serialize(p, jso)).Aggregate((x, y) => $"{x}\n{y}");

Console.WriteLine(data);

{"songId":null,"rank":79,"rankStatus":"undefined","rankStatusValue":null,"title":"Warmth","artist":"Lim Young Woong","album":"Warmth","image":"https://i.scdn.co/image/ab67616d0000b273744791c0e707946a832baa13","trackId":"3vnaEaDxMKdBhqA1t0uAwl","trackUri":"spotify:track:3vnaEaDxMKdBhqA1t0uAwl","danceability":0.354,"valence":0.342}


In [21]:
// Invoke the prompt - RefineResult
var refined = await kernel.InvokeAsync<string>(
    function: prompts["RefineResult"],
    arguments: new KernelArguments()
    {
        { "input", data }
    });

Console.WriteLine(refined);

```json
{
  "tracks": [
    {
      "title": "Warmth",
      "artist": "Lim Young Woong",
      "trackId": "3vnaEaDxMKdBhqA1t0uAwl",
      "trackUri": "spotify:track:3vnaEaDxMKdBhqA1t0uAwl"
    }
  ]
}
```
